In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from imutils import paths
import os
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout,Flatten,Dense,Input
from keras import Sequential
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC 
from scipy.constants import lb
import numpy as np
import cv2 as cv

In [3]:
def matchTemplatePrivate(img):
    METHOD = cv.TM_CCOEFF

    # lê novamente a imagem para evitar dados quebrados
    edged_img = cv.adaptiveThreshold(img, 255,
                                     cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    img2 = img.copy()

    # carrega template para joelho esquerdo e direito
    template_l = cv.imread("../templates/template_L.png", 0)
    template_r = cv.imread("../templates/template_R.png", 0)

    # encontra contornos
    edged_template_l = cv.adaptiveThreshold(template_r, 255,
                                            cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    edged_template_r = cv.adaptiveThreshold(template_l, 255,
                                            cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY_INV, 21, 10)

    w_l, h_l = template_l.shape[::-1]
    w_r, h_r = template_l.shape[::-1]

    # aplica o math template em ambas as imagens de template
    res_l = cv.matchTemplate(edged_img, edged_template_l, METHOD)
    res_r = cv.matchTemplate(edged_img, edged_template_r, METHOD)

    min_val_l, max_val_l, min_loc_l, max_loc_l = cv.minMaxLoc(res_l)
    min_val_r, max_val_r, min_loc_r, max_loc_r = cv.minMaxLoc(res_r)

    # define qual imagem deu melhor match
    if max_val_r > max_val_l:
        top_left = max_loc_r
        bottom_right = (top_left[0] + w_r, top_left[1] + h_r)
    else:
        top_left = max_loc_l
        bottom_right = (top_left[0] + w_l, top_left[1] + h_l)

    return top_left, bottom_right

In [3]:
tf.config.list_physical_devices('GPU')

[]

In [6]:
train_dataset = "C:\\Users\\Windows 10\\Documents\\PUC\\2022_2\\PAI\\KneeXrayData\\ClsKLData\\kneeKL224\\train_preprocessed"
val_dataset = "C:\\Users\\Windows 10\\Documents\\PUC\\2022_2\\PAI\\KneeXrayData\\ClsKLData\\kneeKL224\\val_preprocessed"
test_dataset = "C:\\Users\\Windows 10\\Documents\\PUC\\2022_2\\PAI\\KneeXrayData\\ClsKLData\\kneeKL224\\test_preprocessed"

In [7]:
train_images=list(paths.list_images(train_dataset))
val_images=list(paths.list_images(val_dataset))
test_images=list(paths.list_images(test_dataset))

In [8]:
def processImage(image):
    image = img_to_array(image, dtype='uint8')
    x, y = matchTemplatePrivate(image)
    image = image[x[1]:y[1],x[0]:y[0]]
    image = array_to_img(image)
    image = image.resize((224,224))
    image = img_to_array(image)
    return cv.cvtColor(image,cv.COLOR_GRAY2RGB)

In [9]:
train_data=[]
train_labels=[]
val_data=[]
val_labels=[]
test_data=[]
test_labels=[]

for i in train_images:#adicionar nosso preprocessamento
    label=i.split(os.path.sep)[-2]
    train_labels.append(label)
    image = load_img(i,target_size=(224,224), color_mode="grayscale")
    image = processImage(image)
    train_data.append(image)

for i in val_images:#adicionar nosso preprocessamento
    label=i.split(os.path.sep)[-2]
    val_labels.append(label)
    image = load_img(i,target_size=(224,224), color_mode="grayscale")
    image = processImage(image)
    val_data.append(image)

for i in test_images:#adicionar nosso preprocessamento
    label=i.split(os.path.sep)[-2]
    test_labels.append(label)
    image = load_img(i,target_size=(224,224), color_mode="grayscale")
    image = processImage(image)
    test_data.append(image)


In [10]:
print(test_data[0].shape)
print(len(val_data))
print(len(val_labels))

(224, 224, 3)
1652
1652


In [11]:
train_data=np.array(train_data, dtype='uint8')
train_labels=np.array(train_labels)

val_data=np.array(val_data, dtype='uint8')
val_labels=np.array(val_labels)

test_data=np.array(test_data, dtype='uint8')
test_labels=np.array(test_labels)

In [12]:
train_labels

array(['0', '0', '0', ..., '4', '4', '4'], dtype='<U1')

In [13]:
train_labels_cat = to_categorical(train_labels)

val_labels_cat = to_categorical(val_labels)

test_labels_cat = to_categorical(test_labels)

In [14]:
train_data

array([[[[44, 44, 44],
         [45, 45, 45],
         [45, 45, 45],
         ...,
         [ 7,  7,  7],
         [ 4,  4,  4],
         [ 3,  3,  3]],

        [[44, 44, 44],
         [45, 45, 45],
         [45, 45, 45],
         ...,
         [ 7,  7,  7],
         [ 4,  4,  4],
         [ 3,  3,  3]],

        [[44, 44, 44],
         [45, 45, 45],
         [45, 45, 45],
         ...,
         [ 7,  7,  7],
         [ 4,  4,  4],
         [ 3,  3,  3]],

        ...,

        [[ 7,  7,  7],
         [ 7,  7,  7],
         [ 8,  8,  8],
         ...,
         [17, 17, 17],
         [15, 15, 15],
         [14, 14, 14]],

        [[ 7,  7,  7],
         [ 7,  7,  7],
         [ 7,  7,  7],
         ...,
         [17, 17, 17],
         [15, 15, 15],
         [14, 14, 14]],

        [[ 7,  7,  7],
         [ 7,  7,  7],
         [ 7,  7,  7],
         ...,
         [17, 17, 17],
         [15, 15, 15],
         [14, 14, 14]]],


       [[[ 2,  2,  2],
         [ 3,  3,  3],
         [ 3, 

In [15]:
train_labels_cat

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
train_data, test_data, train_label, test_label = model_selection.train_test_split(train_data, y, train_size=0.80, test_size=0.20, random_state=101)

In [11]:
input_shape = (224, 224, 3)

In [12]:
input_image = Input(shape=input_shape)

In [16]:
clf = SVC(kernel='rbf')

In [16]:
clf.fit(train_data, train_labels)


ValueError: Found array with dim 4. Estimator expected <= 2.